# 0. Import Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import os
import glob
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import math

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras_preprocessing.sequence import TimeseriesGenerator
from keras import Sequential
from keras import layers
from keras.src.layers.core.dense import Dense
from keras.src.layers.reshaping.flatten import Flatten
from keras.src.layers.rnn.lstm import LSTM
from keras.src.layers.rnn.rnn import RNN
from keras.src.optimizers.rmsprop import RMSprop
from keras.src.callbacks.early_stopping import EarlyStopping
from keras.src.callbacks.model_checkpoint import ModelCheckpoint
from keras.src.models.model import model_from_json
from keras import optimizers

# 1. Load data

In [ ]:
# 온도 선택 [N10, 0, 10, 20, 25, 30, 40, 50], temp 변수에 설정

temp = '40'
mode = ['DST', 'FUDS' ,'US06']
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

for m in mode:
    data_dir = os.path.join(parent_dir, f'DB Preprocessing/refined_data/{temp}/{m}')
    file_names = os.listdir(data_dir)
    for file_name in file_names:
        csv_dir = os.path.join(data_dir, file_name)
        if '007' in file_name:
            num = '007'
        else: 
            num = '008'
        globals()['csv_{}'.format(f'{m}_{num}')] = pd.DataFrame(pd.read_csv(csv_dir))
        print(f'csv_{m}_{num} 생성 완료')

In [ ]:
csv_DST_007

# 2. V, I, T / SoC 분리 

In [ ]:
num = ['007','008']

def df2numpy(df):
    np = df.values
    return np

for m in mode:
    for n in num:
        var_name = f'csv_{m}_{n}'
        csv = globals()[var_name]
        globals()[f'input_{m}_{n}'] = df2numpy(csv[['Current(A)', 'Voltage(V)', 'Temperature (C)_1']])
        print(f'input_{m}_{n} 생성 완료')
        globals()[f'output_{m}_{n}'] = df2numpy(csv[['SoC']])
        print(f'output_{m}_{n} 생성 완료')

In [ ]:
print(input_DST_007.shape)
input_DST_007

In [ ]:
print(input_US06_008.shape)
input_US06_008

# 3. Split train and test dataset

In [ ]:
def create_dataset(input, output, look_back=1):
    dataX, dataY = [], []
    for i in range(len(input) - look_back + 1):
        a = input[i:(i + look_back)]
        dataX.append(a)
        dataY.append(output[i + look_back -1])
    dataY = np.reshape(dataY, (len(dataY), 1, 1))
    print(dataY.shape)
    return np.array(dataX), np.array(dataY)

a = [0,1,2,3,4,5,6,7,8,9]
b = [10,11,12,13,14,15,16,17,18,19]
x, y = create_dataset(a, b, 3)
print(x)
print(y)

In [ ]:
# look_back 변수 설정(자연수. 보통 20 설정.)
look_back = 20

dstX_7, dstY_7 = create_dataset(input_DST_007, output_DST_007, look_back)
dstX_8, dstY_8 = create_dataset(input_DST_008, output_DST_008, look_back)
us06X_7, us06Y_7 = create_dataset(input_US06_007, output_US06_007, look_back)
us06X_8, us06Y_8 = create_dataset(input_US06_008, output_US06_008, look_back)
fudsX_7, fudsY_7 = create_dataset(input_FUDS_007, output_FUDS_007, look_back)
fudsX_8, fudsY_8 = create_dataset(input_FUDS_008, output_FUDS_008, look_back)

print(dstX_7.shape)
print(dstY_7.shape)

# 4. Training with Linear Regression

In [ ]:
# 이거랑 이 아래 코드 박스만 수정하셔서 GRU 구현하시면 됩니다.
model = Sequential()
model.add(Flatten(input_shape=(20, 3)))
model.add(Dense(64, input_dim = 5))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse', 'mae'])

In [ ]:
history_dst_7 = model.fit(dstX_7, dstY_7, epochs=100, batch_size=10, verbose=0)
history_us06_7 = model.fit(us06X_7, us06Y_7, epochs=100, batch_size=10, verbose=0)
history_dst_8 = model.fit(dstX_8, dstY_8, epochs=100, batch_size=10, verbose=0)
history_us06_8 = model.fit(us06X_8, us06Y_8, epochs=100, batch_size=10, verbose=0)

In [ ]:
# save model architecture
model_dir = os.path.join(current_dir, 'model', temp)
model_json = model.to_json()
open(f'{model_dir}/{temp}_model.json', 'w').write(model_json)

# save model's learned weights
model.save_weights(f'{model_dir}/{temp}.weights.h5', overwrite=True)

In [ ]:
# Load trained model
json_file = open(f'{model_dir}/{temp}_model.json', "r")
loaded_model_json = json_file.read() 
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# model weight load 
loaded_model.load_weights(f'{model_dir}/{temp}.weights.h5')
print("Loaded model from disk")

In [ ]:
plt.plot(history_dst_7.history['loss'], label='train')

plt.legend() 
plt.show()

In [ ]:
# 007 배터리 예측
fudsY_7_hat = model.predict(fudsX_7)
fudsY_7 = np.reshape(fudsY_7, (fudsY_7.shape[0], 1))
plt.plot(fudsY_7_hat, label='Prediction')
plt.plot(fudsY_7, label='Real data')
plt.legend()
plt.show()

In [ ]:
rmse = math.sqrt(mean_squared_error(fudsY_7, fudsY_7_hat))
mae = mean_absolute_error(fudsY_7, fudsY_7_hat)
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

In [ ]:
# 008 배터리 예측
fudsY_8_hat = model.predict(fudsX_8)
fudsY_8 = np.reshape(fudsY_8, (fudsY_8.shape[0], 1))
plt.plot(fudsY_8_hat, label='Prediction')
plt.plot(fudsY_8, label='Real data')
plt.legend()
plt.show()

In [ ]:
rmse = math.sqrt(mean_squared_error(fudsY_8, fudsY_8_hat))
mae = mean_absolute_error(fudsY_8, fudsY_8_hat)
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)